# Esercizio 2


### Appunti

- Prendo le def
- calcolo i termini più frequenti
- faccio disambiguation
- ottengo synset piu frequenti - che saranno i possibili genus
- confronto 

*DEF di genus:*  Iperonimo più comune dell'oggetto che si pensa che l'interlocutore possa conoscere al meglio; Non sono elementi iper generici o iper specifici, sono ad un livello mediio che più o meno tutti conosciamo (es. mela, pera --> Frutto)

Il punto è non cofrontare tutte le definizioni (100K+) ma confrontarle solo con i genus

### Approccio utilizzato 

In [ ]:
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from collections import Counter
from nltk.wsd import lesk

## Utilities

In [ ]:
def word_sense_disambiguation(list_words, word):
    '''
    Using wordnet's lesk function for disambiguation,
    return the first synset
    '''
    right_synset = lesk(list_words, word)
    return right_synset

def remove_stop_words(row):
    stop_words = set(stopwords.words('english'))
    punctuation = [',', '.', ';', '!', '?', "'", "''", '"', "’", "’’", "`","``"]
    filtered_sentence = [w for w in row if not w.lower() in stop_words and w not in punctuation]
    return filtered_sentence

def calculate_frequency(row):
    '''
    Calculate the frequency of each word in the sentence
    '''
    freq_dict = {}
    row = remove_stop_words(word_tokenize(row))
    for word in row[1:]:
        if word.lower() not in freq_dict:
            freq_dict[word.lower()] = 1
        else:
            freq_dict[word.lower()] += 1
    return freq_dict

def freq_synset(list):
    '''
    Calculate the frequency of each synset in a list of synsets
    '''
    freq_dict = {}
    for word in list:
        if word not in freq_dict:
            freq_dict[word] = 1
        else:
            freq_dict[word] += 1
    return freq_dict

def get_genus(most_freq):
    '''
    Find all the hypernyms of a list of word
    '''
    synsets, genus_list = [], []
    
    # Save the first synset for the most frequent term in definitions
    for word in most_freq:
        # Check if there are ampty values and save the first synset
        if(len(wn.synsets(word[0])) > 0):
            #print(f"--> Word: {word} \n--> Synset: {wn.synsets(word[0])[0]}\n")
            synsets.append(wn.synsets(word[0])[0])

    # for each synset let find all hyponyms and hypernyms and then compare them
    for syn in synsets:
        genus_list.append(syn.hypernyms())
        # genus_list.append(syn.hyponyms())
                
    # list of list to list
    genus = [x for xs in genus_list for x in xs]
    
    return genus

## Execution

### Print the most common word in the definitions with their synset

In [ ]:
with open ('../def.csv', 'r') as f:
    for row in f:
        dict = calculate_frequency(row)
        count = Counter(dict)
        most_freq = count.most_common(5)
        #print(f'\n--> Starting word: *{row.split(",")[0]}*\n')
                        
        genus = get_genus(most_freq)
        print(f'\n{genus}\n')
        
        # Counter most frequent hypernyms and print it
        count_genus = Counter(freq_synset(genus))
        
        # print(count_genus)
        # print(f'{row.split(",")[0]}: {count_genus.most_common(5)}')

        

### Synset Closure

In [59]:
hypo = lambda s: s.hyponyms()
hyper = lambda s: s.hypernyms()

with open ('../def.csv', 'r') as f:
    for row in f:
        dict = calculate_frequency(row)
        count = Counter(dict)
        most_freq = count.most_common(5)
        
        for word in most_freq:
            if(len(wn.synsets(word[0])) > 0):
                syn = wn.synsets(word[0])[0]
                # print(syn.closure(hyper))
                dog = wn.synset('dog.n.01')
                list(dog.closure(hyper))
                    
        print('\n\n\n')
        


# list(dog.closure(hypo, depth=1)) == dog.hyponyms()
# True
# list(dog.closure(hyper, depth=1)) == dog.hypernyms()
# True
# list(dog.closure(hypo))
# list(dog.closure(hyper))



TypeError: 'list' object is not callable

### Fallimentare prova utilizzo lowest common hypernym

In [ ]:
#* Lowest common hypernym approach
#! Don't work: Return too generic genus
lch = []
# check lowest common hypernym
for i in range(len(genus)-1):
        for j in range(len(genus)):
                lch.append(genus[i].lowest_common_hypernyms(genus[j]))     
                # print(lch)